# 0 Load required packages

In [38]:
# External dependencies
import time
from casadi import *
from plot_efficient_frontier import plot_frontier_carlos as plot_frontier

# Internal dependencies
import life_cycle_gradient_descent 
from main import execute_moo_optimization

import htcondor
import htmap

import time
import itertools

import pathlib

In [ ]:
htmap.settings['DELIVERY_METHOD'] = 'assume'

In [24]:
htmap.settings['fixed_input_files'] = 'htmap.TransferPath().cwd()'+'/life_cycle_gradient_descent.py'

In [25]:
bye_path = htmap.TransferPath().cwd()

In [26]:
print(bye_path)

/mnt/d/git_repos/optimal_taxation/python/life_cycle


# 1 Set up life-cycle environment 

### 1.2 Set environment parameter

In [78]:
conf = {
        'tax_setting': 5,       # tax setting: see below for detailed explanation
        'beta': 0.95,           # Utility discounting
        'r_debt': 0.15,         # debt interest rate
        'r': 0.1,
        'T': 60,                # agent's maturity
        'T_ret': 45,            # retirement age
        'L_max': 10,            # maximum labour 
        'gamma': [0.5],         # Agent's utility from consumption
        'eta': [1],             # Agent's disutility from labour
        'plot': False,
        'plot_steps': False,
        'plot_GD_steps': False,
        'h': 1E-6,              # step size finite difference 
        'n_steps_max': 30,      # max number of steps for 1 point on pareto frontier
        'k_max': 20,            # max number of steps for bi-linear search
        'tol': 1E-8,            # tolerance TODO explanation
        'tol_line_search': 1E-8,# tolerance to abort line search
        'serial': False,
        'max_debt': 2,
        'costb': 0.02,
        'borrowing': np.inf,     # np.inf -> allow unlimited borrowing // 0 -> allow no borrowing
        'n_starting_points': 200 # number of starting points for "frontier searching"
        }

# tax settings:

* 1 - capital tax and income tax
* 2 - income tax // NOT possible
* 3 - capital tax = income tax // NOT possible
* 4 - capital tax = income tax, consumption tax
* 5 - capital tax, income tax, consumption tax


# 2 Execute MOO optimization

In [79]:
result_array = execute_moo_optimization_condor(conf)

created map firm-dry-robe with 11 components
[([DM(29.1924), DM(54.3413), DM(59.195), DM(38.0749), DM(47.5222), DM(58.9433), DM(28.797), DM(51.521), DM(52.036), DM(51.0613), DM(50.1826), DM(46.9118)], [DM(96.5539), DM(66.852), DM(52.2855), DM(88.3633), DM(77.5955), DM(53.7673), DM(96.8901), DM(71.8161), DM(70.9803), DM(72.541), DM(73.8769), DM(78.3919)], [0.0015694972026943615, 0.00215657369913184, 0.0026826749933446525, 0.0016905816560363414, 0.0018909645528961162, 0.002617279890771636, 0.001565004728479717, 0.002023993832838483, 0.0020450322289460266, 0.002006137224612608, 0.0019741575374469813, 0.0018741918207035582], [0.25361438141979875, 0.5696489471769782, 0.7021276576673816, 0.3464376300259766, 0.46203877860297055, 0.6894033085740051, 0.24971743601254062, 0.5209220949178324, 0.5292475192478123, 0.5136617340280083, 0.5001868680943709, 0.4537481313347649], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], {'util': [], 'rev': [], 'tax_rates': []}), ([DM(30.9131), DM(48.

# 3 Postprocessing

In [9]:
[pv_rev_d, pv_util_d, tk_d, tl_d, tc_d] = result_array.tolist()
plot_frontier(pv_rev_d, pv_util_d, tk_d, tl_d, tc_d, conf,
                  timestr + "_gamma_" + str(conf['gamma']) + "_eta_" +
                  str(conf["eta"]) + "_tax_setting_" + str(conf["tax_setting"]))

NameError: name 'result_array' is not defined

In [69]:

def execute_moo_optimization_condor(conf):
    """"
        Execute multi-objective optimization problem.
    """


    # extract parameters from configuration dictionary
    gamma = conf["gamma"]
    eta = conf["eta"]
    n_points = conf["n_starting_points"]
    r = conf["r"]
    T = conf["T"]

    # build interest rate vector of length
    r = np.full(T, r)

    # set n_procs
    n_procs = 11
    #n_procs = 1

    # create gamma eta pairs from input
    gammas_etas = itertools.product(gamma, eta)

    # n points per process
    n = n_points // n_procs


    start = time.time()

    for gamma, eta in gammas_etas:
        conf["gamma"] = gamma
        conf["eta"] = eta

        if conf['serial'] or n_procs == 1:
            # SERIAL EXECUTION
            start_point = np.array([0.41, 0.41, 0.41])

            if conf["tax_setting"] == 1:
                start_point[2] = 0

            data = [run_sim_gradient(start_point, 0.005, n, r, conf)]
        else:
            # PARALLEL EXECUTION

            start_points = np.tile(np.array([0.6 / (n_procs + 1) * i + 0.1 for i in range(n_procs)]), (3, 1))

            if conf["tax_setting"] == 1:
                start_points[2, :] = 0

            d_tau = 1/n/n_procs
            arg = [(start_points[:, i], d_tau, n, r, conf) for i in range(n_procs)]
            data = htmap.starmap(life_cycle_gradient_descent.run_sim, args= arg, map_options=htmap.MapOptions(fixed_input_files=[htmap.TransferPath('./life_cycle_gradient_descent.py'), htmap.TransferPath('./auxiliary_functions.py'), htmap.TransferPath('./lifecycle.py')]))
        test = list(data)
        print(test)
        pv_rev_d = np.concatenate([np.array(data[i][0]) for i in range(n_procs)])
        pv_util_d = np.concatenate([np.array(data[i][1]) for i in range(n_procs)])
        tk_d = np.concatenate([np.array(data[i][2]) for i in range(n_procs)])
        tl_d = np.concatenate([np.array(data[i][3]) for i in range(n_procs)])
        tc_d = np.concatenate([np.array(data[i][4]) for i in range(n_procs)])
        GD_steps = [data[i][5] for i in range(n_procs)]

        full_array = np.array([pv_rev_d, pv_util_d, tk_d, tl_d, tc_d])
        elapsed_time = time.time() - start
        print("Elapsed time", elapsed_time, "\n n_procs ", n_procs, "\n n_points ", len(pv_rev_d))


        return full_array, GD_steps


In [47]:
htmap.TransferPath('.')

TransferPosixPath('.')